In [2]:
from nutri_checker.data.loader import load_nutri_data, FINAL_COLUMNS, COLUMNS_QUANTI, COLUMNS_QUALI
from nutri_checker.search.bm25 import NutriIndex
import pandas as pd
from dataclasses import dataclass
from loguru import logger

In [3]:
NUTRIDATA_PATH = "/Users/fweber/code/nutri-checker/data/table_Ciqual_2020_FR_20200707.xls"
NUTRI_MISSING_DATA_PATH = "/Users/fweber/code/nutri-checker/data/missing_values.json"

In [66]:
df = load_nutri_data(NUTRIDATA_PATH, NUTRI_MISSING_DATA_PATH)
index = NutriIndex(df)

In [8]:
import numpy as np

In [ ]:
meals = [
    ("beurre de cacahuètes", 25),
    ("miel", 25),
    ("pain intégral", 75),
    
    ("oeuf blanc", 60*65/100*2),
    ("oeuf jaune", 60*35/100*2),
    ("poireau", 100),
    ("oignon", 50),
    ("lait demi", 5),
    ("moutarde", 5),
    ("huile olive", 10),
    
    ("oignon", 100 * 0.2),
    ("huile olive", 15 * 0.2),
    ("lentille corail", 250 * 0.2),
    ("lait coco", 200 * 0.2),
    ("sauce tomate", 160 * 0.2),
    ("curry", 8 * 0.2),
    ("beurre cacahuètes", 20 * 0.2),
    ("ail cru", 9 * 0.2),
    ("carottes cuites", 145 * 0.2),
    ("navet cuites", 250 * 0.2),
]

In [54]:
alims, qts = list(zip(*meals))

In [67]:
index.retrieve(alims)

,grp_name_fr,ssgrp_name_fr,ssssgrp_name_fr,name_fr,grp_code,ssgrp_code,ssssgrp_code,code,energy_kcal,water_g,...,vit_k2_ug,vit_c_mg,vit_b1_mg,vit_b2_mg,vit_b3_mg,vit_b5_mg,vit_b6_mg,vit_b9_ug,vit_b12_ug,raw_food
948,"fruits, légumes, légumineuses et oléagineux",fruits à coque et graines oléagineuses,-,beurre de cacahuète ou pâte d'arachide,2,205,0,15202,600.0,1.23,...,NaN,0.00,0.160,0.150,14.100,1.090,0.4700,70.00,0.00,True
2528,produits sucrés,"sucres, miels et assimilés",-,miel,7,701,0,31008,300.0,17.60,...,NaN,0.80,0.000,0.069,0.110,0.069,0.0920,2.00,0.00,False
1031,produits céréaliers,pains et assimilés,pains,pain complet ou intégral (à la farine t150),3,302,30201,7110,244.0,36.60,...,NaN,0.00,0.110,0.000,0.000,0.440,0.0870,35.60,0.00,True
1908,"viandes, œufs, poissons et assimilés",œufs,œufs crus,"oeuf, blanc (blanc d'oeuf), cru",4,410,41002,22001,48.1,87.60,...,0.9,0.00,0.004,0.440,0.093,0.200,0.0035,5.50,0.09,True
1909,"viandes, œufs, poissons et assimilés",œufs,œufs crus,"oeuf, jaune (jaune d'oeuf), cru",4,410,41002,22002,307.0,55.00,...,32.1,0.00,0.170,0.380,0.000,2.870,0.2900,159.00,3.03,True
354,"fruits, légumes, légumineuses et oléagineux",légumes,légumes crus,"poireau, cru",2,201,20101,20039,32.3,87.60,...,NaN,18.50,0.065,0.065,0.500,0.130,0.2400,73.00,0.00,True
352,"fruits, légumes, légumineuses et oléagineux",légumes,légumes crus,"oignon, cru",2,201,20101,20034,NaN,89.60,...,NaN,3.90,0.000,0.000,0.000,0.150,0.1000,29.60,0.00,True
1934,produits laitiers et assimilés,laits,laits de vaches liquides (non concentrés),"lait demi-écrémé, uht",5,501,50101,19041,47.0,89.40,...,NaN,1.50,0.060,0.180,0.093,0.410,0.0370,9.49,0.38,False
3011,aides culinaires et ingrédients divers,condiments,-,moutarde,10,1002,0,11013,152.0,67.90,...,NaN,0.30,0.110,0.048,0.570,0.190,0.0710,7.50,0.00,True
2895,matières grasses,huiles et graisses végétales,-,huile d'olive vierge extra,9,902,0,17270,900.0,0.10,...,NaN,0.00,0.000,0.000,0.000,0.000,0.0000,0.00,0.00,False


In [73]:
nutri_daily = index.retrieve(alims)
aa = pd.concat([nutri_daily[["name_fr"]], nutri_daily[COLUMNS_QUANTI] * np.array([qts]).T / 100], axis=1)
aa
#(nutri_daily[COLUMNS_QUANTI] * np.array([qts]).T / 100).fillna(0).sum()

,name_fr,energy_kcal,water_g,prot_g,glucid_g,lipid_g,sugar_g,sugar_fructose_g,sugar_galactose_g,sugar_glucose_g,...,vit_k1_ug,vit_k2_ug,vit_c_mg,vit_b1_mg,vit_b2_mg,vit_b3_mg,vit_b5_mg,vit_b6_mg,vit_b9_ug,vit_b12_ug
948,beurre de cacahuète ou pâte d'arachide,150.000,0.3075,6.35000,4.02500,13.1250,2.62500,0.0300,0.0000,0.03250,...,0.0750,NaN,0.0000,0.04000,0.03750,3.52500,0.27250,0.11750,17.5000,0.0000
2528,miel,75.000,4.4000,0.14000,20.42500,0.0000,19.95000,9.9750,0.7750,8.37500,...,0.0000,NaN,0.2000,0.00000,0.01725,0.02750,0.01725,0.02300,0.5000,0.0000
1031,pain complet ou intégral (à la farine t150),183.000,27.4500,6.89250,33.22500,1.3500,1.65000,0.2250,NaN,0.00000,...,0.0000,NaN,0.0000,0.08250,0.00000,0.00000,0.33000,0.06525,26.7000,0.0000
1908,"oeuf, blanc (blanc d'oeuf), cru",37.518,68.3280,8.42400,0.66300,0.1482,0.55380,0.0546,0.0546,0.26520,...,0.0000,0.702,0.0000,0.00312,0.34320,0.07254,0.15600,0.00273,4.2900,0.0702
1909,"oeuf, jaune (jaune d'oeuf), cru",128.940,23.1000,6.51000,0.45780,11.2140,0.23520,0.0294,0.0294,0.07560,...,0.2940,13.482,0.0000,0.07140,0.15960,0.00000,1.20540,0.12180,66.7800,1.2726
354,"poireau, cru",32.300,87.6000,1.49000,4.90000,0.2500,3.15000,0.6700,NaN,0.65000,...,47.0000,NaN,18.5000,0.06500,0.06500,0.50000,0.13000,0.24000,73.0000,0.0000
352,"oignon, cru",NaN,44.8000,0.55000,3.12500,0.3100,2.40000,0.8500,0.0000,0.80000,...,0.1750,NaN,1.9500,0.00000,0.00000,0.00000,0.07500,0.05000,14.8000,0.0000
1934,"lait demi-écrémé, uht",2.350,4.4700,0.16550,0.24150,0.0775,0.23300,0.0000,0.0000,0.00000,...,0.0100,NaN,0.0750,0.00300,0.00900,0.00465,0.02050,0.00185,0.4745,0.0190
3011,moutarde,7.600,3.3950,0.34600,0.21650,0.5600,0.08300,0.0310,0.0000,0.04300,...,0.0700,NaN,0.0150,0.00550,0.00240,0.02850,0.00950,0.00355,0.3750,0.0000
2895,huile d'olive vierge extra,90.000,0.0100,0.00000,0.00000,9.9900,0.00000,0.0000,0.0000,0.00000,...,5.8100,NaN,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000


In [70]:
index.retrieve(["beurre de cacahuètes", "miel"])[COLUMNS_QUANTI] * np.array([[2, 1]]).T / 100

,energy_kcal,water_g,prot_g,glucid_g,lipid_g,sugar_g,sugar_fructose_g,sugar_galactose_g,sugar_glucose_g,sugar_lactose_g,...,vit_k1_ug,vit_k2_ug,vit_c_mg,vit_b1_mg,vit_b2_mg,vit_b3_mg,vit_b5_mg,vit_b6_mg,vit_b9_ug,vit_b12_ug
948,12.0,0.0246,0.5080,0.322,1.05,0.210,0.0024,0.000,0.0026,0.0,...,0.006,NaN,0.000,0.0032,0.00300,0.2820,0.02180,0.00940,1.40,0.0
2528,3.0,0.1760,0.0056,0.817,0.00,0.798,0.3990,0.031,0.3350,NaN,...,0.000,NaN,0.008,0.0000,0.00069,0.0011,0.00069,0.00092,0.02,0.0


In [7]:
from dataclasses import dataclass, asdict

@dataclass
class Aliment:
    grp_name_fr: str
    ssgrp_name_fr: str
    ssssgrp_name_fr: str
    name_fr: str
    grp_code: int
    ssgrp_code: int
    ssssgrp_code: int
    code: int
    # spec
    energy_kcal: float
    water_g: float
    prot_g: float
    glucid_g: float
    lipid_g: float
    sugar_g: float
    sugar_fructose_g: float
    sugar_galactose_g: float
    sugar_glucose_g: float
    sugar_lactose_g: float
    sugar_maltose_g: float
    sugar_saccharose_g: float
    amidon_g: float
    fiber_g: float
    alcool_g: float
    acid_sat_g: float
    acid_monoinsat_g: float
    acid_polyinsat_g: float
    cholesterol_g: float
    salt_g: float
    calcium_g: float
    iron_mg: float
    mg_mg: float
    vit_a_ug: float
    vit_d_ug: float
    vit_e_ug: float
    vit_k1_ug: float
    vit_k2_ug: float
    vit_c_mg: float
    vit_b1_mg: float
    vit_b2_mg: float
    vit_b3_mg: float
    vit_b5_mg: float
    vit_b6_mg: float
    vit_b9_ug: float
    vit_b12_ug: float
    # technical
    raw_food: bool
    
    def __post_init__(self):
        if not self.raw_food:
            logger.warning(f"Aliment {self.name_fr} is not raw food")
    
    @classmethod
    def from_row(cls, row):
        return cls(**row.to_dict())
    
    
    @classmethod
    def from_name_in_nutridata(cls, name: str, nutri_index: NutriIndex):
        if not any(cook_mode in name.lower() for cook_mode in ["cuit", "cuite", "cuisiné", "cuisinée"]):
            name = name + " " + "cru"
            
        aliment = nutri_index.retrieve(name, k=1).iloc[0].to_dict()
        return cls(**aliment)
    
    def get_nutritive_spec(self) -> pd.Series:
        return pd.Series(asdict(self))
        

In [8]:
@dataclass
class Ingredient:
    aliment: Aliment
    quantity_g: float
    
    def get_nutritive_spec(self) -> pd.Series:
        nutritive_unit_row = self.aliment.get_nutritive_spec()
        return pd.concat([nutritive_unit_row[COLUMNS_QUALI], nutritive_unit_row[COLUMNS_QUANTI] * 3])
    
    
@dataclass
class Pan:
    name: str
    weight: float
    

In [9]:
class IndustrialDish(Aliment):
    def __post_init__(self):
        pass
    
    def set_nutritive_spec(self, nutritive_details_from_package: dict) -> None:
        for key, value in nutritive_details_from_package.items():
            if not hasattr(self, key):
                logger.warning(f"Nutritive detail {key} not found in Aliment")
                continue
            setattr(self, key, value)
        

In [10]:
from typing import List

class Dish:
    def __init__(self, name, ingredients: List[Ingredient], total_weight_g=None, final_weight_g=None):
        self.name = name
        self.ingredients = ingredients
        self.total_weight_g = total_weight_g or sum(ing.quantity_g for ing in ingredients)
        # final weight after water evaporation. Default to total weight
        self.final_weight_g = final_weight_g if final_weight_g else self.total_weight_g

    def __str__(self):
        return f"{self.name} ({len(self.ingredients)} ingredients)"
    
    def __repr__(self):
        return f"{self.name} ({len(self.ingredients)} ingredients)"
    
    def __len__(self):
        return len(self.ingredients)

    def __iter__(self):
        return iter(self.ingredients)
    
    def __getitem__(self, idx):
        return self.ingredients[idx]
    
    def __setitem__(self, idx, value):
        self.ingredients[idx] = value
    
    def __delitem__(self, idx):
        del self.ingredients[idx]
    
    def __contains__(self, item):
        return item in self.ingredients
    
    def __add__(self, other):
        return Dish(self.name + " & " + other.name, self.ingredients + other.ingredients)
    
    def __iadd__(self, other):
        self.ingredients += other.ingredients
        return self
    
    def get_portion(self, portion_g):
        return Dish(self.name, [Ingredient(ing.aliment, portion_g * ing.quantity_g / self.final_weight_g) for ing in self.ingredients], total_weight_g=portion_g)
    
    def get_nutri(self):
        return pd.DataFrame(data=[ing.get_nutritive_spec() for ing in self.ingredients])

In [14]:
d = Dish("test", ingredients=[Ingredient(Aliment.from_name_in_nutridata("Pomme", nutri_index), 300), Ingredient(Aliment.from_name_in_nutridata("Banane", nutri_index), 100)])

In [16]:
d.get_nutri()

,grp_name_fr,ssgrp_name_fr,ssssgrp_name_fr,name_fr,grp_code,ssgrp_code,ssssgrp_code,code,energy_kcal,water_g,...,vit_k1_ug,vit_k2_ug,vit_c_mg,vit_b1_mg,vit_b2_mg,vit_b3_mg,vit_b5_mg,vit_b6_mg,vit_b9_ug,vit_b12_ug
0,"fruits, légumes, légumineuses et oléagineux",fruits,fruits crus,"pomme, pulpe, crue",2,204,20401,13050,113.625,260.1,...,1.8,0.0,12.00,0.057,0.084,0.273,0.213,0.111,0.0,0.0
1,"fruits, légumes, légumineuses et oléagineux",fruits,fruits crus,"banane, pulpe, crue",2,204,20401,13005,271.500,227.4,...,0.0,0.0,21.48,0.162,0.000,1.170,0.930,0.540,57.0,0.0


In [ ]:
class Menu:
    def __init__(self, name, dishes: List[Dish]):
        self.name = name
        self.dishes = dishes
    
    def __str__(self):
        return f"{self.name} ({len(self.dishes)} dishes)"
    
    def __repr__(self):
        return f"{self.name} ({len(self.dishes)} dishes)"
    
    def __len__(self):
        return len(self.dishes)

    def __iter__(self):
        return iter(self.dishes)
    
    def __getitem__(self, idx):
        return self.dishes[idx]
    
    def __setitem__(self, idx, value):
        self.dishes[idx] = value
    
    def __delitem__(self, idx):
        del self.dishes[idx]
    
    def __contains__(self, item):
        return item in self.dishes
    
    def __add__(self, other):
        return Menu(self.name + " & " + other.name, self.dishes + other.dishes)
    
    def __iadd__(self, other):
        self.dishes += other.dishes
        return self
    
    def get_portion(self, portion_g):
        return Menu(self.name, [dish.get_portion(portion_g) for dish in self.dishes])
    
    def get_nutri(self):
        return pd.DataFrame(data=[dish.get_nutri() for dish in self.dishes]).sum()

In [14]:
a = Dish("a", [
    Ingredient(Aliment.from_row(df.iloc[0]), 100),
    Ingredient(Aliment.from_row(df.iloc[10]), 100)
    ])

In [15]:
a.get_nutri()

grp_name_fr           fruits, légumes, légumineuses et oléagineuxfru...
ssgrp_name_fr                                            légumeslégumes
ssssgrp_name_fr                                légumes cruslégumes crus
name_fr                           avocat, pulpe, crucéleri branche, cru
grp_code                                                              4
ssgrp_code                                                          402
ssssgrp_code                                                      40202
code                                                              33027
energy_kcal                                                       222.6
water_g                                                           163.7
prot_g                                                             2.19
glucid_g                                                           3.24
lipid_g                                                            20.6
sugar_g                                                         